In [1]:
val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"

interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._


path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import plotly._, element._, layout._, Plotly._

import ujson.Value

import sys.process._


In [2]:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await

import $ivy.$                                         

import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.Action

import edu.colorado.plv.bounder.RunConfig

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.ExperimentsDb

import edu.colorado.plv.bounder.BounderUtil

import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.GetResult

import slick.jdbc.SQLActionBuilder

import scala.concurrent.Await

In [3]:
val expDir = File("/Users/shawnmeier/Documents/source/bounder/experiments/twoAppsAllDeref")
val baseDir = File("/Users/shawnmeier/Documents/data/fdroid")

expDir: File = /Users/shawnmeier/Documents/source/bounder/experiments/twoAppsAllDeref
baseDir: File = /Users/shawnmeier/Documents/data/fdroid

In [4]:
def createConfigsFromList():Unit = {
    val mode = Driver.SampleDeref
    val baseConfig = RunConfig(limit = -1, samples = 10)
    case class ApkSample(apk:String, pkg:String, outDir:File)
    val apkSamples = (expDir / "appSample.txt")
        .contentAsString
        .split("\n")map{v => 
            val pkg = v.split(":")(1)
            val apk = v.split(":")(0)
            val apkDirCfg = apk.toString.replace(baseDir.toString,"${baseDir}")
            val outDirCfg = "${baseDirOut}/" + pkg
            val outDir = expDir / pkg
//             val getSample = outDir / "getSample.json"
//             if(getSample.exists())getSample.delete()
            val cfg = baseConfig.copy(apkPath=apkDirCfg, outFolder=Some(outDirCfg))
//             getSample.append(write[RunConfig](cfg))
            outDir.createIfNotExists(true)
            Action(Driver.MakeAllDeref, Some(expDir.toString), Some(baseDir.toString),
                   cfg)
        }
    apkSamples.foreach{action => 
        println(s"action: ${action}")
        Driver.runAction(action)
    }
//     apkSamples.foreach{println}
//     apkSamples.map{cfg => Driver.sampleDeref(cfg, cfg.apkPath, cfg.outFolder)}
}
//createConfigsFromList()

action: Action(MakeAllDeref,Some(/Users/shawnmeier/Documents/source/bounder/experiments/twoAppsAllDeref),Some(/Users/shawnmeier/Documents/data/fdroid),RunConfig(${baseDir}/itkach.aard2/0.43/apk/itkach.aard2_43.apk,Some(${baseDirOut}/itkach.aard2),None,TopSpecSet,List(),-1,10,,1200),None,None)
action: Action(MakeAllDeref,Some(/Users/shawnmeier/Documents/source/bounder/experiments/twoAppsAllDeref),Some(/Users/shawnmeier/Documents/data/fdroid),RunConfig(${baseDir}/com.serwylo.lexica/0.12.0/apk/com.serwylo.lexica_1200.apk,Some(${baseDirOut}/com.serwylo.lexica),None,TopSpecSet,List(),-1,10,,1200),None,None)


defined function createConfigsFromList

In [2]:
expDir.glob("**/*.json").toList.size

cmd2.sc:1: not found: value expDir
val res2 = expDir.glob("**/*.json").toList.size
           ^Compilation Failed

: 

In [3]:
val db = new ExperimentsDb(Some("/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"))

Initializing database


db: ExperimentsDb = edu.colorado.plv.bounder.ExperimentsDb@2bf2e93f

In [ ]:
// Upload data of APK files to database for experiments
def uploadApks() = {
    def appList = (expDir / "appSample.txt").contentAsString.split("\n")
        .map{v => 
            val spl = v.split(":")
            val name = spl(0).replace(baseDir.toString,"")
            val path = spl(0)
            (name,path)
        }
    appList.foreach(a => db.uploadApk(a._1,File(a._2)))
}
uploadApks()

In [ ]:
//db.downloadApk("/com.gpl.rpg.AndorsTrail/0.7.5/apk/com.gpl.rpg.AndorsTrail_46.apk", File("/Users/shawnmeier/Desktop/testDownload.apk"))

In [6]:
// Create empty run config
val emptyCfg = db.createConfig(
    File("/Users/shawnmeier/Documents/source/bounder/soot_hopper/specs/emptySpec.txt"),
    File("/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"),
    "Empty spec set, commit 3f2ca32f03ad9100796e1c2679ca130455c6e635"
)

emptyCfg: Int = 2

In [ ]:
// push experiment .json files to database
def pushExp() = {
    val experiments = expDir.glob("**/*.json").toList
    experiments.foreach(v => db.createJob(v, Some("empty1"), emptyCfg))
}
//pushExp()

In [8]:
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

def pauseJobs() = {
    val pauseJobsQ = sql"""update jobs set status='paused' where status='new';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def resumeJobs() = {
    val pauseJobsQ = sql"""update jobs set status='new' where status='paused';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def clearInputs() = {
    val clearInputsQ = sql"""delete from apks where apkname like 'jar_%' or apkname like 'spec_%';"""
    Await.result(db.db.run(clearInputsQ.as[StrRes]), 30 seconds)
    val clearInputsTableQ = sql"""delete from inputs;"""
    Await.result(db.db.run(clearInputsTableQ.as[StrRes]), 30 seconds)
}
def resetJobs() = {
    val resetJobsQ = sql"""update jobs set inputid=2, stderr='',stdout='',owner='',started=NULL,ended=NULL,status='new';"""
    Await.result(db.db.run(resetJobsQ.as[StrRes]), 30 seconds)
}

Await.result(db.db.run(sql"""select count(*) from results;""".as[Count]), 30 seconds)

defined class Count
defined class StrRes
getCountResult: AnyRef with GetResult[Count] = <function1>
getStrResult: AnyRef with GetResult[StrRes] = <function1>
defined function pauseJobs
defined function resumeJobs
defined function clearInputs
defined function resetJobs
res7_8: Vector[Count] = Vector(Count(661))

In [9]:
resumeJobs()

res8: Vector[StrRes] = Vector(322)